In [2]:
import pyspark
from pyspark.sql import SparkSession

# Start notebook
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("victor_hwasser_applicationA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/23 17:35:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/23 17:35:52 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [ ]:
# A1

from pyspark import SparkConf, SparkContext
#conf = spark_session.conf
sc = spark_session.sparkContext 

text_sv = sc.textFile('hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.sv')
text_en = sc.textFile('hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.en')

# Uncomment to inspect the data!
#text_sv.take(10)

count_sv = text_sv.count()
count_en = text_en.count()
count_sv_par = text_sv.getNumPartitions()
count_en_par = text_en.getNumPartitions()

if count_sv == count_en:
    print("Both text documents have the same amount of lines:", count_sv)
    print("Number of partitions:", count_sv_par, "and", count_en_par)
else:
    print("The documents doesn't have the same amount of lines, something's wrong!", count_sv, "vs", count_en)

22/02/23 17:36:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/23 17:36:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/23 17:36:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/23 17:37:01 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/23 17:37:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/02/23 17:37:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
# A2

def pre_text(text):
    # Make each sentence lower case
    text_lower = text.map(lambda x: x.lower())
    # split each sentence into words
    #!! It would make more sence to use flatMap, but this wouldn't word for A 2.3
    words = text_lower.map(lambda x: x.split())
    return words

# get rdd with processed text
words_sv = pre_text(text_sv)
words_en = pre_text(text_en)

# A.2.2 Inspect 10 entries from each of your RDDs to verify your pre-processing.
first_sv = words_sv.take(10)
first_en = words_en.take(10)

print(first_sv)
print(first_en)

# A.2.3 Verify that the line counts still match after the pre-processing.
count_sv = words_sv.count()
count_en = words_en.count()

if count_sv == count_en:
    print("Both text documents have the same amount of lines:", count_sv)
else:
    print("The documents doesn't have the same amount of lines, something's wrong!", count_sv, "vs", count_en)



In [ ]:
# A3

# word count, basically a map-reducer
def count_words(text):
    # Make each sentence lower case
    text_lower = text.map(lambda x: x.lower())
    # split each sentence into words and flatten
    words = text_lower.flatMap(lambda x: x.split())
    # reduce words
    counted_words = words.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x + y)
    return counted_words

# The algorithm here is to do a MapReduce, sort the values and take the top ten ones
counted_words_en = count_words(text_en)
sorted_words_en = counted_words_en.sortBy(lambda x: x[1])
top_en = sorted_words.top(10)
print(top_en)

# Doing the same thing to swedish words
counted_words_sv = count_words(text_sv)
sorted_words_sv = counted_words_sv.sortBy(lambda x: x[1])
top_sv = sorted_words.top(10)
print(top_sv)



In [ ]:
# A4

words_sv
words_en

sv_1 = words_sv.zipWithIndex()
en_1 = words_en.zipWithIndex()
sv_2 = sv_1.map(lambda x,y: (y,x))
en_2 = en_1.map(lambda x,y: (y,x))
step_3 = sv_2.zip(en_2)
step_4 = step_3.filter(lambda (x,y): x == []
step_5 = 
